In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle
from itertools import product
import requests
import os
import random
from fake_useragent import UserAgent
from itertools import cycle

In [ ]:
# Getting proxies
proxies = pd.read_csv('/Users/macbook/Downloads/Webshare 100 proxies.txt')
print('Number of proxies:', len(proxies))
proxies = proxies['Proxy'].values
proxy_pool = cycle(proxies)

In [ ]:
def start_browsing():
    
    ua = UserAgent()
    user_agent = ua.random
    proxy = next(proxy_pool)
    
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": proxy,
        "ftpProxy": proxy,
        "sslProxy": proxy,
        "proxyType": "MANUAL"
    }
    chrome_options = webdriver.ChromeOptions()
    chrome_options.headless=False
    chrome_options.add_argument('--user-agent=%s' % user_agent)
    
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 2})
    path = '/Users/macbook/Downloads/Chrome Driver'
    browser =  webdriver.Chrome(os.path.join(path, 'chromedriver'),chrome_options=chrome_options)
    browser.maximize_window()
    
    
    url = 'https://www.allianz.de/gesundheit/tierkrankenversicherung/rechner/#/'
    browser.get(url)
    

    # Cookies
    path = '/html/body/div[1]/div[2]/div[4]/div[2]/div/button'
    element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, path)))
    element.click()

    # Input ZIP
    path = '//*[@formcontrolname="zip"]'
    element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, path)))
    element.clear()
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys('99998')
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.3)
    # Street
    path = '//*[@formcontrolname="street"]'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.3)
    # House number
    path = '//*[@formcontrolname="streetNumber"]'
    browser.find_element_by_xpath(path).send_keys('1')
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)

    sleep(0.7)  
    # Input birthdate
    path = '//*[@formcontrolname="birthdate"]'
    browser.find_element_by_xpath(path).send_keys('01.12.1990')
    sleep(0.7) 

    # Choose pet
    path = '//*[@formcontrolname="animalCategory"]'
    browser.find_element_by_xpath(path).click()
    sleep(0.4)
    browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
    sleep(0.2)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)
    
    if browser.find_element_by_xpath(path).text == 'Meine Katze':
        pass
    else:
        print('Error when choosing an animal')
        browser.quit()

    sleep(0.7) 
    # Birthdate
    path = '//*[@formcontrolname="animalBirthDate"]'
    try:
        browser.find_element_by_xpath(path).send_keys('01.12.2020')
    except:
        pass
    sleep(0.2)
    browser.find_element_by_xpath(path).send_keys('01.12.2020')
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys(Keys.ESCAPE)
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)

    sleep(0.5)
    # Wurde Ihr Tier kastriert oder sterilisiert?
    # Choosing 'Ja'
    path = '//*[@formcontrolname="sterilized"]//nx-radio[1]'
    browser.find_element_by_xpath(path).click()

    sleep(0.5)
    # Wie wird Ihre Katze gehalten?
    # Choosing "ausschließlich in der Wohnung"
    path = '//*[@formcontrolname="catHousingType"]//nx-radio[1]'
    browser.find_element_by_xpath(path).click()

    sleep(0.5)
    # War oder ist Ihr Tier wegen Krankheiten, Fehlentwicklungen oder Unfällen beim Tierarzt in Behandlung?
    # Choosing "Nein"
    path = '//*[@formcontrolname="preExistingDiagnosis"]//nx-radio[2]'
    browser.find_element_by_xpath(path).click()
    
    return browser

In [ ]:
def get_price(raw_price, clean=True):
    if clean:
        raw_price = re.findall(r'(?<=\().+(?=\))', raw_price)[0]
    price = re.sub(r'\,', '.', raw_price)
    price = re.sub(r'[^\.\d]', '', price)
    price = float(price)
    return price

### Getting Prices for every race

In [ ]:
browser = start_browsing()
actions = ActionChains(browser)

In [ ]:
race_prices = pd.DataFrame(columns=['Race'])

In [ ]:
races_dict = {}
for i in range(1,160):

    path = '//*[@formcontrolname="animalRace"]'
    element = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, path)))
    element.click()

    sleep(0.2)
    path = '/html/body/div[5]/div[10]/div/div/div[2]/nx-dropdown-item[{}]'.format(i)
    browser.find_element_by_xpath(path).click()

    sleep(0.2)
    path = '//*[@formcontrolname="animalRace"]'
    race = browser.find_element_by_xpath(path).text
    
    races_dict[race] = i

In [ ]:
missed_races = [x for x in races_dict.keys() if x not in race_prices['Race'].unique()]
missed_races_idx = [races_dict[x] for x in missed_races]

In [ ]:
for i in missed_races_idx:

    print('\n#', i)

    '''
    Selecting race
    '''

    try:

        for _ in range(1):
            actions.send_keys(Keys.PAGE_UP).perform()

        path = '//*[@formcontrolname="animalRace"]'
        element = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, path)))
        element.click()

        sleep(0.2)
        path = '/html/body/div[5]/div[10]/div/div/div[2]/nx-dropdown-item[{}]'.format(i)
        browser.find_element_by_xpath(path).click()

        sleep(0.2)
        path = '//*[@formcontrolname="animalRace"]'
        race = browser.find_element_by_xpath(path).text

        if race in race_prices['Race'].unique():
            continue

        print(race)

        # Getting Tarif data
        path = '/html/body/div[3]/div/div[4]/div/div/div/div/div/app-root/div/page-data/div/div[2]/form/div/div/div[5]/button'
        browser.find_element_by_xpath(path).click()

    except:
        print('UNKNOWN ERROR')
        continue


    '''
    Getting prices
    '''

    try:

        # Waiting until the price is presented

        path = '//*[@id="product_TM001"]'
        element = WebDriverWait(browser, 30).until(EC.visibility_of_element_located((By.XPATH, path)))

        path = '/html/body/div[3]/div/div[4]/div/div/div/div/div/app-root/div/page-offer/div/div[2]/div/module-table/div/nx-comparison-table/div/nx-comparison-table-flex-row[1]/div[2]'
        header_price = browser.find_element_by_xpath(path).text

        basic, smart, komfort, premium = [header_price.split('\n')[i] for i in [1,5,9,13]]


        page_df = pd.DataFrame({'Basic':basic, 
                                'Smart':smart,
                                'Komfort':komfort,
                                'Premium':premium}, index=[0])

        page_df['Race'] = race
        race_prices = pd.concat([race_prices, page_df], ignore_index=True)
        print('GOT DATA')

    except:
        print('PAGE ERROR')
        browser.back()
        continue


    """
    Back to input data
    """
    browser.back()

    # Save the dataset
    #race_prices.to_csv('ALIANZ_RACE_DATA_STEP2.csv')

### Getting race price groups

In [ ]:
for col in race_prices.columns[:-2]:
    race_prices[col] = race_prices[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    race_prices[col] = race_prices[col].astype(float)

In [ ]:
race_prices['Group'] = ''
for col in race_prices.columns[:-2]:
    race_prices['Group'] = race_prices['Group'] + '-' + race_prices[col].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(race_prices['Group'])
race_prices['Group Num'] = enc.transform(race_prices['Group'])
race_prices = race_prices.drop(columns='Group')

In [ ]:
race_prices['Group Num'].nunique()

In [ ]:
#race_prices.to_csv('ALIANZ_RACE_DATA_STEP2.csv')

### Getting price groups for zip codes

In [ ]:
zip_prices = pd.read_csv('ALIANZ_ZIP_DATA_STEP1.csv', index_col=[0])
zip_prices['Basic'].nunique()

In [ ]:
for col in zip_prices.columns[1:]:
    zip_prices[col] = zip_prices[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    zip_prices[col] = zip_prices[col].astype(float)

In [ ]:
zip_prices['Group'] = ''
for col in zip_prices.columns[1:]:
    zip_prices['Group'] = zip_prices['Group'] + '-' + zip_prices[col].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(zip_prices['Group'])
zip_prices['Group Num'] = enc.transform(zip_prices['Group'])
zip_prices = zip_prices.drop(columns='Group')

In [ ]:
group_zips = zip_prices.drop_duplicates(['Group Num'])['ZIP'].values
zip_prices['Group Num'].nunique()